<a href="https://colab.research.google.com/github/addy8403/pro3/blob/main/QnAChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 901.6 kB/s eta 0:00:00


In [ ]:
pip install pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.9 MB/s eta 0:00:00


In [ ]:
import os
import openai


In [ ]:
file_path='/content/drive/MyDrive/Colab Notebooks/Hidden-API/api_key.txt'
with open(file_path,'r') as file:
  api_key=file.read().strip()

In [ ]:
openai.api_key=api_key

In [ ]:
!pip install faiss-cpu numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.7 MB/s eta 0:00:00


In [ ]:
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams

def extract_text_from_pdf(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path, laparams=laparams)
    return text

pdf_path = '/content/dsa.pdf'
text = extract_text_from_pdf(pdf_path)



In [ ]:
def split_text_into_chunks(text, max_tokens=1500):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) > max_tokens:
            chunks.append(" ".join(current_chunk[:-1]))
            current_chunk = [word]

    chunks.append(" ".join(current_chunk))
    return chunks


chunks = split_text_into_chunks(text)


In [ ]:
len(chunks)

201

In [ ]:
response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input="Hi")
print(response['data'][0]['embedding'])

[-0.036372922360897064, -0.007158639375120401, -0.033741358667612076, -0.028634557500481606, -0.026836754754185677, 0.034575119614601135, -0.012441311031579971, -0.007816530764102936, 0.0019297059625387192, -0.0026511065661907196, 0.024700237438082695, -0.0024540650192648172, -0.005774462129920721, -0.0029849375132471323, 0.006670106202363968, -0.003022391814738512, 0.03379346802830696, -0.001511195907369256, 0.021052522584795952, -0.009028092958033085, -0.021703900769352913, 0.010369930416345596, 0.006302077788859606, 0.007093501277267933, -0.012252411805093288, 0.0008191072847694159, 0.005872168578207493, -0.009835801087319851, -0.0030451901257038116, -0.024713264778256416, 0.01078029815107584, -0.013822231441736221, -0.024491796270012856, -0.014108837582170963, 0.0024442942813038826, -0.018968114629387856, 0.0005850184825249016, -0.01137305237352848, 0.018134351819753647, -0.009959562681615353, 0.013118743896484375, -0.011294886469841003, -0.009158368222415447, -0.00971203949302435,

In [ ]:
import time
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']


def convert_chunks_to_embeddings(chunks):
    embeddings = []
    for i, chunk in enumerate(chunks):
        print(i)
        embedding = get_embeddings(chunk)
        embeddings.append(embedding)

    return embeddings


In [ ]:
embeddings=convert_chunks_to_embeddings(chunks)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [ ]:
people=[]
for i in range(len(chunks)):
  new_dict={"chunk":chunks[i], "embedding":embeddings[i]}
  people.append(new_dict)

In [ ]:
import numpy as np
import faiss

import os

def get_top_3(query):
  chunks_with_embeddings =people

  def build_faiss_index(chunks_with_embeddings):
      # Extract embeddings and create a numpy array
      embeddings = np.array([item['embedding'] for item in chunks_with_embeddings]).astype('float32')

      # Get the dimension of the embeddings
      dimension = embeddings.shape[1]


      index = faiss.IndexFlatL2(dimension)

      # Add the embeddings to the index
      index.add(embeddings)

      return index

  def get_query_embedding(query):
      response = openai.Embedding.create(
          model="text-embedding-ada-002",
          input=query
      )
      return np.array(response.data[0].embedding).astype('float32').reshape(1, -1)

  def retrieve_top_chunks(index, chunks_with_embeddings, query, k=3):
      # Get the query embedding
      query_vector = get_query_embedding(query)

      # Search the index
      distances, indices = index.search(query_vector, k)

      # Retrieve the top k chunks
      top_chunks = [chunks_with_embeddings[i]['chunk'] for i in indices[0]]

      return list(zip(top_chunks, distances[0]))

  # Build the index
  faiss_index = build_faiss_index(chunks_with_embeddings)

  # Example usage
  results = retrieve_top_chunks(faiss_index, chunks_with_embeddings, query)

  return results


In [ ]:
def QnA():
  query=input("Enter your question about data structures and algorithms:")
  top=get_top_3(query)
  top_chunks = [chunk for chunk, _ in top]
  combined_data=" ".join(top_chunks)


  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
      {"role": "system", "content": f"You are a college student studying for a test. \
      Consider the content given below as your textbook and study it so that \
      you will be able to answer questions from it. \
      Here are your guidelines to answer questions: \
      1.)Your answer should only contain content from the textbook and irrelevant question \
      or anything outside the textbook should not be answered. \
      2.)As a student if the question is out of the textbook you have the right \
      to not answer them . \
      3.)Your test scores will be high only if you use the \
      content given in the textbook. \
      4.)Using outside content will result in negtive marking so remember that as well. \
      textbook: ```{combined_data}``` "},
      {"role": "user", "content": f"Question: {query}"}
      ]
  )
  return response.choices[0].message["content"]

In [ ]:
res = QnA()
print(res)

Enter your question about data structures and algorithms:what are graphs?
A graph consists of nodes (vertices or points) and edges (lines or links) connecting the nodes. Graphs can be simple, weighted, directed, or undirected. They can be represented using different data structures like pointer-based implementation in Java. Graph theory involves studying relationships between graphs, such as isomorphism, subgraphs, supergraphs, and containment.


In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display
query=input("Enter your question about data structures and algorithms:")
top=get_top_3(query)
top_chunks = [chunk for chunk, _ in top]
combined_data=" ".join(top_chunks)
context=[
      {"role": "system", "content": f"You are a senior college professor from a reputed university. \
      Your task is to give a concise lecture on the textbook given below and be ready to answer questions \
      from the students based on the textbook. Make the student feel comfortable so that he can freely ask you questions and clear his doubts   \
      Here are your guidelines to answer questions: \
      1.)Your answer should only contain content from the textbook and irrelevant question \
      or anything outside the textbook should not be answered. \
      2.)As a teacher if the question is out of the textbook you have the right \
      to not answer them as they will distract the students who are present for learning. \
      3.)Your answers should only contain content from the textbook as the students will \
      assessed on the textbook\
      4.)Using content from outside the book will confuse the students and decrease your reputation so keep that in mind  . \
      textbook: ```{combined_data}``` "},
      {"role": "user", "content": f"Question: {query}"}
      ]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Enter your question about data structures and algorithms:Given an absolute path for a Unix-style file system, which begins with a slash '/', transform this path into its simplified canonical path.  In Unix-style file system context, a single period '.' signifies the current directory, a double period ".." denotes moving up one directory level, and multiple slashes such as "//" are interpreted as a single slash. In this problem, treat sequences of periods not covered by the previous rules (like "...") as valid names for files or directories.  The simplified canonical path should adhere to the following rules:  It must start with a single slash '/'. Directories within the path should be separated by only one slash '/'. It should not end with a slash '/', unless it's the root directory. It should exclude any single or double periods used to denote current or parent directories. Return the new path.     Example 1:  Input: path = "/home/"  Output: "/home"  Explanation:  The trailing slash s

<ipython-input-45-e97edc341c46>:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

<ipython-input-45-e97edc341c46>:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
<ipython-input-45-e97edc341c46>:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


In [ ]:
messages =  context.copy()
response = get_completion_from_messages(messages, temperature=0)
print(response)

In [ ]:
# 1. field expert
# 2. task
# 3. instruction about response
# 4. given context

In [ ]:
pip install llama-index

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import SimpleVectorStore
from llama_index.embeddings import OpenAIEmbedding
from llama_index.schema import Document

In [ ]:

documents = [Document(text=item['chunk'], embedding=item['embedding']) for item in chunks_with_embeddings]


embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Create a vector store
vector_store = SimpleVectorStore()

# Create an index from the documents
index = VectorStoreIndex.from_documents(
    documents,
    embedding_model=embed_model,
    vector_store=vector_store
)

# Function to retrieve top 3 chunks for a given query
def retrieve_top_chunks(query, top_k=3):

    query_engine = index.as_query_engine(similarity_top_k=top_k)

    response = query_engine.query(query)

    top_chunks = [node.node.text for node in response.source_nodes]
    return top_chunks

